In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
Before ReLU: tensor([[-4.6991e-01, -1.8445e-01, -4.0548e-01, -2.1362e-01,  1.8044e-01,
         -4.7134e-01, -1.8531e-01, -4.0668e-01,  5.1854e-01,  1.5316e-01,
          1.0220e+00,  1.4289e-03,  5.5671e-02, -4.6817e-01, -6.2308e-02,
         -4.1305e-01, -5.4105e-01,  5.0500e-05,  4.4998e-01,  7.3657e-01],
        [-5.5988e-01, -1.3176e-01, -1.7313e-01, -5.0137e-01, -2.0800e-02,
         -3.5407e-01, -1.1645e-01, -2.8075e-01,  3.2904e-01,  4.3399e-01,
          7.3477e-01, -1.0862e-01, -1.5432e-01, -3.3739e-01,  2.0735e-01,
         -2.2910e-01, -3.0415e-01,  1.0704e-01,  5.5411e-01,  9.0119e-01],
        [-5.5038e-01,  1.1767e-01, -6.9583e-01, -2.8890e-01, -2.6658e-03,
         -5.3064e-01, -2.2326e-01, -1.9790e-01,  2.2703e-02,  4.7593e-01,
          1.1038e+00,  1.9418e-01, -5.3308e-02, -1.7906e-02,  7.6617e-02,
         -1.1862e-01, -9.2643e-01,  4.0455e-02,  3.4654e-01,  5.4209e-01]],
       grad_fn=<AddmmBackward0